In [11]:
import pandas as pd
import os
import numpy as np

def preprocess_pm25_data(years):
    """
    Membaca data PM2.5 per jam dari file Excel, membersihkan tipe data, 
    mengagregasi ke rata-rata harian, menggabungkan semua tahun, 
    melakukan reindexing, dan menerapkan interpolasi linear.
    """
    all_daily_data = []
    
    # --- JALUR ABSOLUT DATA (Harap pastikan jalur ini benar di PC Anda) ---
    BASE_PATH = r"C:\Users\user\OneDrive\IPB\Thesis\02. Development\00. Dataset\dataset pm25\dlhdki\Data PM2,5 Tahun 2024 (per jam)\kebunjeruk"
    # ----------------------------------------------------------------------
    
    TIME_COL = 'Date / Time'
    TARGET_COL = 'PM2,5'
    OUTPUT_COL = 'PM2.5 Daily Mean'
    
    print(f"--- Memulai Pra-pemrosesan Data Kebun Jeruk ---")

    for year in years:
        file_name = f"{year}.xlsx"
        file_path = os.path.join(BASE_PATH, file_name)
        
        if not os.path.exists(file_path):
            print(f"❌ ERROR: File '{file_path}' tidak ditemukan. Melewati tahun {year}.")
            continue
            
        print(f"\n1. Membaca dan Membersihkan file: {file_name}")
        
        try:
            df_hourly = pd.read_excel(file_path, engine='openpyxl')
            
            # --- Pembersihan Tipe Data (Mengatasi Koma dan String) ---
            df_hourly[TARGET_COL] = df_hourly[TARGET_COL].astype(str).str.replace(',', '.', regex=False)
            df_hourly[TARGET_COL] = pd.to_numeric(df_hourly[TARGET_COL], errors='coerce') 
            # -----------------------------------------------------------
            
            # 2. Agregasi ke Harian (Daily Mean)
            df_hourly[TIME_COL] = pd.to_datetime(df_hourly[TIME_COL], errors='coerce')
            df_hourly = df_hourly.set_index(TIME_COL)
            
            df_daily = df_hourly[TARGET_COL].resample('D').mean().to_frame()
            df_daily.columns = [OUTPUT_COL]
            
            print(f"   -> Berhasil diagregasi: {len(df_daily)} hari data harian untuk tahun {year}.")
            all_daily_data.append(df_daily)
            
        except KeyError:
            print(f"❌ ERROR: Kolom '{TARGET_COL}' atau '{TIME_COL}' tidak ditemukan di file {file_name}. Periksa ejaan kolom.")
        except Exception as e:
            print(f"❌ ERROR: Gagal memproses file {file_name}. Detail: {e}")
            
    if not all_daily_data:
        print("\n--- ERROR: Tidak ada data yang berhasil diproses. Program selesai. ---")
        return

    # --- TAHAP PEMEROSESAN AKHIR: Kontinuitas & Interpolasi ---
    print("\n--- 2. Penggabungan, Reindexing, dan Interpolasi ---")
    df_final = pd.concat(all_daily_data)
    
    # 3. REINDEXING: Memastikan seluruh rentang waktu 2022-2024 ada
    start_date = '2022-01-01'
    end_date = '2024-12-31'
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D', name='Date')
    
    # Reindex df_final. Hari yang hilang akan ditambahkan sebagai NaN.
    df_final = df_final.reindex(full_date_range)
    print(f"   -> Data di-reindex. Total hari dalam rentang: {len(df_final)} hari.")

    # A. Pengecekan Data Kosong Awal
    missing_count_initial = df_final[OUTPUT_COL].isnull().sum()
    print(f"   A. Jumlah data kosong PM2.5 sebelum interpolasi (NaN): {missing_count_initial} baris.")
    
    # B. Interpolasi Linear (Pengisian Data Kosong)
    if missing_count_initial > 0:
        # Interpolasi Linear: mengisi gap dengan garis lurus antara dua titik data terdekat
        df_final[OUTPUT_COL].interpolate(method='linear', inplace=True)
        print("   B. Interpolasi Linear berhasil diterapkan.")
    else:
        print("   B. Interpolasi tidak diperlukan.")
    
    # C. Pengecekan Data Kosong Akhir (Verifikasi)
    missing_count_final = df_final[OUTPUT_COL].isnull().sum()
    
    # Hanya ada NaN yang tidak bisa diisi (jika missing data ada di awal/akhir seri)
    if missing_count_final > 0:
        # Jika masih ada data kosong, itu berarti berada di awal atau akhir time series
        # Kita bisa mengisi bagian ini dengan nilai rata-rata dari data yang ada
        df_final[OUTPUT_COL].fillna(df_final[OUTPUT_COL].mean(), inplace=True)
        print(f"   C. Tersisa {missing_count_final} baris kosong (di ujung seri), diisi dengan mean.")

    # 4. Menyimpan hasil akhir ke CSV
    output_filename = 'kebun_jeruk_pm25_daily_interpolated_2022_2024.csv'
    df_final.to_csv(output_filename)
    
    print("\n--- Proses Selesai ---")
    print(f"✅ Data akhir tersimpan ke file: {output_filename}")
    print(f"Total baris data harian akhir: {len(df_final)}")
    
if __name__ == "__main__":
    years_to_process = [2022, 2023, 2024]
    preprocess_pm25_data(years_to_process)

--- Memulai Pra-pemrosesan Data Kebun Jeruk ---

1. Membaca dan Membersihkan file: 2022.xlsx
   -> Berhasil diagregasi: 365 hari data harian untuk tahun 2022.

1. Membaca dan Membersihkan file: 2023.xlsx
   -> Berhasil diagregasi: 365 hari data harian untuk tahun 2023.

1. Membaca dan Membersihkan file: 2024.xlsx
   -> Berhasil diagregasi: 366 hari data harian untuk tahun 2024.

--- 2. Penggabungan, Reindexing, dan Interpolasi ---
   -> Data di-reindex. Total hari dalam rentang: 1096 hari.
   A. Jumlah data kosong PM2.5 sebelum interpolasi (NaN): 282 baris.
   B. Interpolasi Linear berhasil diterapkan.

--- Proses Selesai ---
✅ Data akhir tersimpan ke file: kebun_jeruk_pm25_daily_interpolated_2022_2024.csv
Total baris data harian akhir: 1096


C:\Users\user\AppData\Local\Temp\ipykernel_6808\2431184339.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final[OUTPUT_COL].interpolate(method='linear', inplace=True)


In [10]:
df_final.info()

NameError: name 'df_final' is not defined